In [1]:
from Go2Py.sim.mujoco import Go2Sim
from Go2Py.robot.model import Go2Model
import pinocchio as pin 
import numpy as np
import time

In [2]:
model = Go2Model()
model.get

In [3]:
robot = Go2Sim()

In [4]:
robot.step()

In [7]:
omega = robot.getIMU()['gyro']
def hat(w):
    return np.array([[0., -w[2], w[1]],
              [w[2], 0., -w[0]],
              [-w[1], w[0], 0.]])

In [ ]:
robot.standUpReset()
while True:
    time.sleep(0.001)
    robot.setCommands(robot.standing_q, np.zeros(12), np.ones(12)*100, np.ones(12)*0.1, np.zeros(12))
    robot.step()    
    trans, quat = robot.getPose()
    state = robot.getJointStates()
    quat = pin.Quaternion(np.hstack([quat[1:],quat[0]]))
    R = quat.matrix()
    T = np.eye(4)
    T[0:3,0:3] = R
    vel = np.zeros(6)
    model.update(state['q'], state['dq'],T,vel)

    J = model.getInfo()['J']['FR_foot'][:,6:]
    nle = model.getInfo()['nle'][6:]
    # tau = (robot.data.qfrc_smooth+robot.data.qfrc_constraint)[6:]
    tau = state['tau_est'].squeeze()
    print(np.linalg.pinv(J.T)[0:3]@(tau - nle))

In [ ]:
question: What exactly is the difference between the local world aligned and local jacobians in the pinocchio? Why it fits my expectations for the FR3 and not the Go2?

In [ ]:
robot.getJointStates()


In [ ]:
J[3:,:].T